# A. Project Name:  IMDb Successful Movie.
- **Student Name:** Eduardo Galindez.
- **Coding Dojo Bootcamp:** Data Science.
  - **Stack:** Data Enrichment.
- **Date:** September 23th, 2022.

# B. Project Objective
For Part E of the project, we will be using our MySQL database from Part C to answer meaningful questions for the stakeholder. They want we to use our hypothesis testing and statistics knowledge to answer 3 questions about what makes a successful movie.

# C. Project Statement


### Questions to Answer:
The stakeholder's first question is: 
1. Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?
    - They want we to perform a statistical test to get a mathematically-supported answer.
    - They want we to report if we found a significant difference between ratings.
        - If so, what was the p-value of you analysis?
        - And which rating earns the most revenue?
    - They want we to prepare a visualization that supports your finding.

2. It is then up to we to think of 2 additional hypotheses to test that your stakeholder may want to know.

3. Some example hypotheses we could test:
- Do movies that are over 2.5 hours long earn more revenue than movies that are 1.5 hours long (or less)?
- Do movies released in 2020 earn less revenue than movies released in 2018?
- How do the years compare for movie ratings?
- Do some movie genres earn more revenue than others?
- Are some genres higher rated than others? etc.

# D. Project Development

## 1.- Libraries & Functions

In [2]:
# Libraries.
import numpy as np
import pandas as pd
import tmdbsimple as tmdb 
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm_notebook

## 2.-  Data & Connection

### 2.2.- Loading data.
- The data to work with is from Part D.

In [ ]:
# Load basics.
title_basics_df = pd.read_csv('./D')

In [4]:
# Load combine dataframe from Part D:
df = pd.read_csv('./Data/tmdb_results_combined.csv.gz')
#tmbd_data = tmbd_df[['imdb_id','budget', 'revenue','certifcation']]
df.head(5)

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certifcation
0,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
1,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.100,8.0,NaN
2,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,0.000,0.0,NaN
3,tt0118694,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,cn,花樣年華,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.110,1984.0,PG
4,tt0118852,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,49511.0,en,Chinese Coffee,...,0.0,99.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,There's a fine line between friendship and bet...,Chinese Coffee,0.0,6.851,47.0,R


### 2.3.- Get Budget, Revenue and Certification per movie tested in Section 2.1.

#### 2.3.1.- The Avengers

#### 2.3.2.- The Notebook

## 3.- Visual Data Exploration

### 3.1.- How many movies had at least some valid financial information (values > 0 for budget OR revenue)?
Please exclude any movies with 0's for budget AND revenue from the remaining visualizations.

**Notes:**
- We identified there is still missing data in our 'certification' column, even after filtering.
- The impact of these missing data has been reduced to 12.6%.

### 3.2.- How many movies are there in each of the certification categories (G/PG/PG-13/R)?

### 3.3.- What is the average revenue per certification category?

### 3.4.- What is the average budget per certification category?

### 3.5.- What is the average ROI (Return of Investment) per certification category?
- In Section 2.3, we calculated the ROI of The Avengers and The Notebook movies. 
- In this Section we are going to analyze if those financial performance are usual in movies, in order to evaluate which type of movie are more profitable related with MPAA Rating (certification).

# E. Conclusions

- In 2000 and 2001, rating R accounted for almost 67% of the movies made, but was not among the top three in profitability.
- During the same time period, PG movies accounted for 8% of films made, and they were identified as the most profitable.
- There were 8.86% of projects with positive ROI, which raised suspicions about the data's reliability. There might need to be a more in-depth study.